In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [2]:
import torch
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [3]:
tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/ruT5-large")

model = AutoModelForSeq2SeqLM.from_pretrained("detoxified_release_large", local_files_only=True)

In [4]:
from sklearn.model_selection import train_test_split

df = pd.read_csv("detoxified.csv")
df["toxic"] = df["toxic"].apply(lambda x: x.replace('!!', '!'))
df["toxic"] = df["toxic"].apply(lambda x: x.replace('!!', '!'))
df["toxic"] = df["toxic"].apply(lambda x: x.replace('!!', '!'))
df["toxic"] = df["toxic"].apply(lambda x: x.replace('!!', '!'))
df["toxic"] = df["toxic"].apply(lambda x: x.replace('!!', '!'))
train_df, eval_df = train_test_split(df, test_size=0.2)

In [5]:
from torch.utils.data import Dataset, DataLoader

class ToxicDataset(Dataset):
    def __init__(self, input_ids, attention_mask, labels, decoder_attention_mask):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.labels = labels
        self.decoder_attention_mask = decoder_attention_mask
        
    def __len__(self):
        return int(self.input_ids.size(dim=0))
    
    def __getitem__(self, i):
        return {"input_ids": self.input_ids[i], "attention_mask": self.attention_mask[i], "labels": self.labels[i], "decoder_attention_mask": self.decoder_attention_mask[i]}
    

In [6]:
prefix = tokenizer(train_df["detoxified"].values.tolist(), return_tensors="pt", padding=True, truncation=True, max_length=512, pad_to_multiple_of=8)
train_input_ids, train_attention_mask = prefix["input_ids"], prefix["attention_mask"]
suffix = tokenizer(train_df["toxic"].values.tolist(), return_tensors="pt", padding=True, truncation=True, max_length=512)
train_labels, train_decoder_attention_mask = suffix["input_ids"], suffix["attention_mask"]

prefix = tokenizer(eval_df["detoxified"].values.tolist(), return_tensors="pt", padding=True, truncation=True, max_length=512, pad_to_multiple_of=8)
eval_input_ids, eval_attention_mask = prefix["input_ids"], prefix["attention_mask"]
suffix = tokenizer(eval_df["toxic"].values.tolist(), return_tensors="pt", padding=True, truncation=True, max_length=512)
eval_labels, eval_decoder_attention_mask = suffix["input_ids"], suffix["attention_mask"]

In [7]:
train_dataset = ToxicDataset(train_input_ids, train_attention_mask, train_labels, train_decoder_attention_mask)
eval_dataset = ToxicDataset(eval_input_ids, eval_attention_mask, eval_labels, eval_decoder_attention_mask)

In [8]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

In [9]:
args = Seq2SeqTrainingArguments(
    output_dir = "detoxified_model_large_exclam",
    evaluation_strategy = "steps",
    per_device_train_batch_size = 4,
    per_device_eval_batch_size = 8,
    gradient_accumulation_steps = 8,
    num_train_epochs = 1,
    save_strategy = "steps",
    save_steps = 500,
    fp16 = True,
    eval_steps = 250,
    dataloader_num_workers = 4,
    group_by_length = True,
    generation_max_length=256
)

trainer = Seq2SeqTrainer(
    model = model,
    args = args,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,
    tokenizer = tokenizer
)

Using amp half precision backend


In [10]:
trainer.train()

/home/jupyter-black_samorez/.conda/envs/megatron/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 39544
  Num Epochs = 1
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 8
  Total optimization steps = 1235


Step,Training Loss,Validation Loss
250,No log,0.119911
500,0.133900,0.118985


***** Running Evaluation *****
  Num examples = 9887
  Batch size = 8
***** Running Evaluation *****
  Num examples = 9887
  Batch size = 8
Saving model checkpoint to detoxified_model_large_exclam/checkpoint-500
Configuration saved in detoxified_model_large_exclam/checkpoint-500/config.json
Model weights saved in detoxified_model_large_exclam/checkpoint-500/pytorch_model.bin
tokenizer config file saved in detoxified_model_large_exclam/checkpoint-500/tokenizer_config.json
Special tokens file saved in detoxified_model_large_exclam/checkpoint-500/special_tokens_map.json
Copy vocab file to detoxified_model_large_exclam/checkpoint-500/spiece.model


KeyboardInterrupt: 

In [12]:
tokenizer.decode(model.generate(tokenizer(["Если кто не знает -- ray marching это техника рендеринга сцен, в которой лучи итеративно \"прощупывают\" сцену с помощью лишь функции расстояния до ближайшего объекта. Если алгоритм рассматривает луч в какой-то точке и знает расстояние до сцены, то это значит, что можно пройти это расстояние в направлении луча и ничего не задеть. Алгоритм проходит ровно это расстояние, и приступает к следующей итерации."], return_tensors="pt").input_ids.cuda())[0].cpu(), skip_special_tokens=True)

'если кто не знает -- ray marching это техника рендеринга сцен, в которой'

In [11]:
model.save_pretrained("detoxified_release_large_exclam")

Configuration saved in detoxified_release_large_exclam/config.json
Model weights saved in detoxified_release_large_exclam/pytorch_model.bin


In [37]:
test_input = "Если кто не знает -- ray marching это техника рендеринга сцен, в которой лучи итеративно \"прощупывают\" сцену с помощью лишь функции расстояния до ближайшего объекта."

tokenizer.decode(model.generate(tokenizer([test_input], return_tensors="pt").input_ids.cuda(),
                                temperature=1.2,
                                do_sample = True,
                                length_penalty = 0.9,
                                max_length=512,
                                repetition_penalty=2.0)[0].cpu(), skip_special_tokens=True)

'Если кто не знает -- эйнмаххрен это техника рендеринга сцен, в которой лучи итеративным способом накладывают свое отпечаток только при этом условии. Я так понимаю что роль формы позволяет быть более или ровнее (до 5 м/с) чем после неё жеманной сцены а уж тем меньше её убытков по сравнению с размеренными ттлпми? Изгои пуки-ды из космоса есть даже как раз такого размера для этого нужно потратить определенную суммарное пространство от 0 до 6м3 вплоть лет! Свидетельницей того момента надо ставить сопроыскательным образом перед зрителяями: те почему их там нет чтобы им удобно было делать такое дебильное фото'